## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd
data = pd.read_csv('credit_score_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

##### 'children'
- 47 строк со значением "-1", т.к. из них 29 человек имеют статус "женат / замужем", предположительно это некорректно введенная еденица.
- 76 строк со значение "20", т.к. количество детей аномальное, и между 5 и 20 нет вариантов, это предоложительно неправильно введенная двойка. 

##### 'days_employed'
- положительные значения встречаются у тех, кто сейчас не работает - пенсионеры и безработные. Отрицательные значения у тех, кто работает. У положительных значений вместо дней указаны часы. Все нужно привести к одному формату.
- 2174 строк с пропусками. Т.к. по этим строкам средний дефолт равен среднему по всей выборке около 8%, логично заполнить пропуски средними значениями.

##### 'dob_years'
- 101 человек с возрастом 0. Среди этих людей попадаются и сотрудники и пенсионеры, что похоже на техническую ошибку в программе или выгрузке. При дальнейшей обработке эти значениея заменим на средний возраст по выборке.
- нужно переименовать в понятное название full_years

##### 'education'
- Часть категорий находятся в разном регистре, все будет переведоно в нижний регистр.

##### 'gender'
- одна строка со значением 'XNA', из остальных данных вывод о поле сделать не получается, строку прийдется удалить.

##### 'total_income'
- 2174 строк с пропусками. Т.к. по этим строкам средний дефолт равен среднему по всей выборке около 8%, логично заполнить пропуски медианными.

##### 'purpose'
- Цели вписаны "от руки", а значит их нужно лемматизировать и разбить на категории.

##### 'education_id', 'family_status', 'family_status_id', 'income_type', 'debt'
- Замечаний нет

<font color='green'>Данные загружены и изучены, отлично подмечены основные проблемы.</font>

### Шаг 2. Предобработка данных

### Обработка пропусков

Возьмем столбцы с пропусками 'total_income', 'days_employed' и попробуем найти закономерность с другими столбцами. Препдологается что мы найдем связь между пропусками в какой-либо группе в столбце. Это поможет понять причину пропусков.

In [3]:
columns_with_na = ['total_income', 'days_employed' ]
target_columns = ['children', 'education', 'family_status', 'gender', 'income_type', 'purpose']
data['counter'] = 1
def investigetion_of_na(column_with_na, target_column):
    pvt = pd.pivot_table(data, values=['counter',column_with_na],\
                             index=target_column, aggfunc='count')
    pvt.columns = ["all_rows", "not_na"]
    pvt['percent_of_na'] =  ((pvt["all_rows"] - pvt["not_na"]) / pvt["all_rows"] * 100).astype(int)
    pvt = pvt.sort_values("all_rows", ascending = False)
    print(pvt)
for column_with_na in columns_with_na:
    print('='*15,'Investigation about this column:',column_with_na,'='*15)
    for target_column in target_columns:
        print('-'*25,column_with_na, 'VS', target_column, '-'*25)
        investigetion_of_na(column_with_na, target_column)    

=============== Investigation about this column: total_income ===============
------------------------- total_income VS children -------------------------
          all_rows  not_na  percent_of_na
children                                 
 0           14149   12710             10
 1            4818    4343              9
 2            2055    1851              9
 3             330     294             10
 20             76      67             11
-1              47      44              6
 4              41      34             17
 5               9       8             11
------------------------- total_income VS education -------------------------
                     all_rows  not_na  percent_of_na
education                                           
среднее                 13750   12342             10
высшее                   4718    4222             10
СРЕДНЕЕ                   772     705              8
Среднее                   711     646              9
неоконченное высшее       668

                                        all_rows  not_na  percent_of_na
purpose                                                                
свадьба                                      797     721              9
на проведение свадьбы                        777     685             11
сыграть свадьбу                              774     693             10
операции с недвижимостью                     676     615              9
покупка коммерческой недвижимости            664     597             10
операции с жильем                            653     579             11
покупка жилья для сдачи                      653     588              9
операции с коммерческой недвижимостью        651     581             10
жилье                                        647     587              9
покупка жилья                                647     595              8
покупка жилья для семьи                      641     570             11
строительство собственной недвижимости       635     560        

Пропуски в столбцах 'total_income', 'days_employed' составляют примерно 10% от всех строк, и по всем категориям они распределены более-менее равномерно. Это похоже на техническую ошибку, возможно при выгрузке данных. Что бы не потеять информацию по этим клиентам, принято решение заполнить данные исходя из средних/медианных значений по категориям.

In [4]:
income_tbl = data.groupby('income_type')['total_income'].median()
def fill_income(row):
    if row['total_income'] != row['total_income']:
        return income_tbl[row['income_type']]
    else:
        return row['total_income']
data['total_income'] = data.apply(fill_income, axis=1)

In [5]:
data.loc[data['children'] == 20,'children'] = 2
data.loc[data['children'] == -1,'children'] = 1

data = data.rename(columns={'dob_years':'full_years'})
data.loc[data['full_years'] < 18,'full_years'] = int(data[data['full_years'] > 18]['full_years'].mean())

data['education'] = data['education'].str.lower()

data = data.loc[data['gender'] != 'XNA'] 

### Вывод

- удалена только одна строка, где небыл указан пол. Данные установить не удалось.
- full_years пропуски были заполненны средним возрастом по выборке.
- total_income пропуски были заполненны медианным значением по каждой группе.

### Замена типа данных

In [6]:
def prepare_years_employed(days_employed):
    if days_employed < 0:
        return int(days_employed / 365 * -1)
    elif days_employed > 0:
        return int(days_employed / (365 * 24))
    else:
        0
data['years_employed'] = data['days_employed'].apply(prepare_years_employed)
data = data.fillna(value={'years_employed':data['years_employed'].mean()})
data.loc[:,'years_employed'] = data['years_employed'].astype(int)
data = data.drop(columns=['days_employed'])

data.loc[:,'total_income'] = data['total_income'].astype(int)

### Вывод

- рабочий стаж был преобразован в годы, для удобства дальнейшей обработки
- общий доход бы округлен до целых чисел

### Обработка дубликатов

In [7]:
print("Всего дубликатов -", data.duplicated().sum())
data = data.drop_duplicates()

Всего дубликатов - 71


### Вывод

- Найден 71 дубликат. Единственное что их объединяет - это отсутствие информации о стаже и доходе. Но эта проблема более общая, т.к. без стажа и дохода всего 2174 строки. 
- Больше похоже на ошибку при выгрузке данных, поэтому дубликаты просто удалены.

### Лемматизация

In [8]:
from pymystem3 import Mystem
from collections import Counter

def debt_rate(column):
    debt_rate = data.groupby(column)['debt'].sum() / data.groupby(column)['debt'].count() * 100
    return debt_rate.sort_values(ascending=False)

m = Mystem()

str_purposes = ''
for cell in data['purpose']:
    str_purposes += ' ' + cell
lst_purposes = m.lemmatize(str_purposes)
print(Counter(lst_purposes))
print('Выбираем категории :','жилье','автомобиль','недвижимость', 'образование','свадьба')

def purpose_group(purpose):   
    purpose_list = ['жилье','автомобиль','недвижимость', 'образование','свадьба']
    for word in m.lemmatize(purpose):
        if word in purpose_list:
            return word
        
data['purpose_group'] = data['purpose'].apply(purpose_group)

print('='*30,'Разбивка групп по количеству','='*30)
print(data['purpose_group'].value_counts(normalize=True))
print('='*30,'Разбивка групп по просрочке','='*30)
print(debt_rate('purpose_group'))

Counter({' ': 55022, 'недвижимость': 6350, 'покупка': 5896, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})
Выбираем категории : жилье автомобиль недвижимость образование свадьба
============================== Разбивка групп по количеству ==============================
недвижимость    0.295996
жилье           0.207896
автомобиль      0.200718
образование     0.187060
свадьба         0.108330
Name: purpose_group, dtype: float64
============================== Разбивка групп по просрочке ==============================
purpose_group
автомобиль      9.3590

### Вывод

- после лемматизации получилось 5 групп целей.
- Недвижимость и жилье - группы похожие, и часто пересекающиеся. Но принял решение не объединять, т.к. слово недвижимость чаще используется для коммерческой недвижимости, а жилье для частной.

### Категоризация данных

In [9]:
min_age = data['full_years'].min()-1
max_age = data['full_years'].max()+1
years_list = [min_age] + [int(i) for i in list(data['full_years'].quantile([.2,.4,.6,.8]))] +[max_age]

def age_group(full_years):
    for i in range(len(years_list)):
        if full_years < years_list[i]:
            return str(years_list[i-1]) + '-' + str(years_list[i])
        
min_income = data['total_income'].min()-1
max_income = data['total_income'].max()+1
lst_of_income = [min_income] + [int(i) for i in list(data['total_income'].quantile([.2,.4,.6,.8]))] +[max_income]
def total_income_group(total_income):
    for i in range(len(lst_of_income)):
        if total_income < lst_of_income[i]:
            return str(lst_of_income[i-1]) + '-' + str(lst_of_income[i])

def children_group(children):
    if children > 0:
        return 1
    return 0
        
data['age_group'] = data['full_years'].apply(age_group)        
data['total_income_group'] = data['total_income'].apply(total_income_group)
data['children_group'] = data['children'].apply(children_group) 


### Вывод

Для разбивки выборки по возрасту и доходу было два основных критерия:
- Группы должны получиться примерно одинаковые по количеству людей
- Просрочка по группам должна меняться последовательно, либо убывать от группе к группе, либо возрастать.

Столбец с детьми решил разбить на две категории, 1 - наличие детей, 0 - отсутствие. Это сделано потому что процент людей у которых два и более детей - всего около 10%, что может дать ненужную погрешность, а показатели просрочки по всей группе, где есть дети очень похожи.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [10]:
print('='*30,'Разбивка групп по количеству','='*30)
print(data['children_group'].value_counts())
print('='*30,'Разбивка групп по просрочке','='*30)
print(debt_rate('children_group'))

============================== Разбивка групп по количеству ==============================
0    14090
1     7363
Name: children_group, dtype: int64
============================== Разбивка групп по просрочке ==============================
children_group
1    9.208203
0    7.544358
Name: debt, dtype: float64


### Вывод

Можно сделать вывод, что зависомость между наличием детей и возвратом кредита в срок есть. Група клиентов, у которых есть дети, попадают в просрочку на 22 % чаще чем те у кого нет детей. В абсолютных числах у клиентов без детей просрочка в среднем 7,5%, с детьми 9,2%, разница составляет 1,7%.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [11]:
print('='*30,'Разбивка групп по количеству','='*30)
print(data['family_status'].value_counts())
print('='*30,'Разбивка групп по просрочке','='*30)
print(debt_rate('family_status'))

============================== Разбивка групп по количеству ==============================
женат / замужем          12339
гражданский брак          4150
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64
============================== Разбивка групп по просрочке ==============================
family_status
Не женат / не замужем    9.750890
гражданский брак         9.349398
женат / замужем          7.545182
в разводе                7.112971
вдовец / вдова           6.569343
Name: debt, dtype: float64


### Вывод

Формально можно сазать что зависимость между семейным положением и возвратом кредита в срок есть. Но сдесь нужно обратить внимание на то, что также просматривается зависимость от количества клиентов в группе. Т.е. чем больше людей в группе, тем выше просрочка. Этот факт может говорить о наличии погрешности в этом показателе. Что бы разобраться в этом, нужно смотреть как этот показатель проявит себя при сочетании с другими показателями.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [12]:
print('='*30,'Разбивка групп по количеству','='*30)
print(data['total_income_group'].value_counts())
print('='*30,'Разбивка групп по просрочке','='*30)
print(debt_rate('total_income_group'))

============================== Разбивка групп по количеству ==============================
214622-2265605    4291
20666-98536       4291
132131-161332     4291
98536-132131      4290
161332-214622     4290
Name: total_income_group, dtype: int64
============================== Разбивка групп по просрочке ==============================
total_income_group
132131-161332     8.739222
98536-132131      8.414918
161332-214622     8.414918
20666-98536       8.016779
214622-2265605    6.991377
Name: debt, dtype: float64


### Вывод

Явной лиенейной зависимости в этом показателе не наблюдается. Разница цифр по просрочке в пределах 1 - 1,5% что  больше похоже на случайный разброс. Однако в сочетании с отстальными параметрами, этот показатель может себя проявить, а значит окончательный вывод можно будет сделать если посмотрим как этот показатель работает в сочетании с остальными.

- Как разные цели кредита влияют на его возврат в срок?

In [13]:
print('='*30,'Разбивка групп по количеству','='*30)
print(data['purpose_group'].value_counts())
print('='*30,'Разбивка групп по просрочке','='*30)
print(debt_rate('purpose_group'))

============================== Разбивка групп по количеству ==============================
недвижимость    6350
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_group, dtype: int64
============================== Разбивка групп по просрочке ==============================
purpose_group
автомобиль      9.359034
образование     9.220035
свадьба         8.003442
недвижимость    7.464567
жилье           6.905830
Name: debt, dtype: float64


### Вывод

Показатель цели кредита влияет на его возврат в срок. Что выражется не только в цифрах разницы по группам до 2,5% между жильем(6,9%) и автомобилем(9,4%), но и в логике. Вполне понятно почему кредиты на жилье возвращают лучше, чем кредиты на автомобили.

- Зависимость между другими показателями и дефолтом.

In [14]:
print('='*30,'Разбивка групп по количеству','='*30)
print(data['education'].value_counts())
print('='*30,'Разбивка групп по просрочке','='*30)
print(debt_rate('education'))

print('='*30,'Разбивка групп по количеству','='*30)
print(data['gender'].value_counts())
print('='*30,'Разбивка групп по просрочке','='*30)
print(debt_rate('gender'))

print('='*30,'Разбивка групп по количеству','='*30)
print(data['income_type'].value_counts())
print('='*30,'Разбивка групп по просрочке','='*30)
print(debt_rate('income_type'))

print('='*30,'Разбивка групп по количеству','='*30)
print(data['age_group'].value_counts())
print('='*30,'Разбивка групп по просрочке','='*30)
print(debt_rate('age_group'))

============================== Разбивка групп по количеству ==============================
среднее                15172
высшее                  5250
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64
============================== Разбивка групп по просрочке ==============================
education
начальное              10.992908
неоконченное высшее     9.152086
среднее                 8.990245
высшее                  5.295238
ученая степень          0.000000
Name: debt, dtype: float64
============================== Разбивка групп по количеству ==============================
F    14174
M     7279
Name: gender, dtype: int64
============================== Разбивка групп по просрочке ==============================
gender
M    10.262399
F     7.012840
Name: debt, dtype: float64
============================== Разбивка групп по количеству ==============================
сотрудник          11084
компаньон           5077
пенсион

### Вывод

Нужно отметить что есть дополнительные категории, которые значительно влияют на возврат кредита в срок.
- Женщины имеют просрочку 7%, а мужчины 10,2%, что на 45% выше чем женщины.
- Люди с высшим образование имеют просрочку 5,3%, а со средним образование 9%, что почти в два раза выше.
- Пенсионеры попадают в просрочку в 5,6%, а сотрудники в 9,6%, что тоже значительно выше.

Эти три показателя имеют одни из самых высоких предскательных потенциалов, однако для окончательных выводов необходимо построить общую модель, где будет учитываться как все эти показатели сочетыются вместе.

In [15]:
pvt = pd.pivot_table(data, values=['counter','debt'], index=['children_group','purpose_group', 'total_income_group' ], aggfunc='sum')
pvt['pd'] = pvt['debt'] / pvt['counter'] * 100
pvt = pvt.sort_values('pd')
pvt

counter  debt         pd
children_group purpose_group total_income_group                          
1              свадьба       214622-2265605          163     6   3.680982
0              жилье         214622-2265605          577    31   5.372617
               недвижимость  98536-132131            837    48   5.734767
               жилье         20666-98536             603    35   5.804312
1              жилье         132131-161332           302    18   5.960265
0              свадьба       214622-2265605          294    18   6.122449
               жилье         161332-214622           571    35   6.129597
               свадьба       161332-214622           305    19   6.229508
1              жилье         214622-2265605          335    21   6.268657
0              свадьба       20666-98536             318    21   6.603774
               автомобиль    214622-2265605          556    37   6.654676
               недвижимость  214622-2265605          811    54   6.658446
                             20666-98536             885    61   6.892655
               жилье         98536-132131            592    41   6.925676
               недвижимость  132131-161332           767    54   7.040417
1              жилье         20666-98536             298    22   7.382550
0              недвижимость  161332-214622           872    67   7.683486
               образование   214622-2265605          505    39   7.722772
               автомобиль    132131-161332           541    42   7.763401
1              жилье         98536-132131            280    22   7.857143
               недвижимость  214622-2265605          459    37   8.061002
                             161332-214622           430    35   8.139535
0              автомобиль    20666-98536             582    48   8.247423
               образование   161332-214622           513    43   8.382066
                             98536-132131            555    49   8.828829
1              недвижимость  132131-161332           498    44   8.835341
0              образование   132131-161332           519    46   8.863198
               свадьба       98536-132131            338    30   8.875740
1              свадьба       132131-161332           180    16   8.888889
0              жилье         132131-161332           558    50   8.960573
1              автомобиль    20666-98536             254    23   9.055118
                             214622-2265605          327    30   9.174312
               свадьба       98536-132131            151    14   9.271523
               образование   161332-214622           269    25   9.293680
               недвижимость  98536-132131            407    38   9.336609
                             20666-98536             384    36   9.375000
0              образование   20666-98536             550    52   9.454545
               автомобиль    98536-132131            623    59   9.470305
1              жилье         161332-214622           344    33   9.593023
               образование   20666-98536             268    26   9.701493
0              свадьба       132131-161332           275    27   9.818182
1              свадьба       161332-214622           151    15   9.933775
               образование   132131-161332           319    32  10.031348
                             214622-2265605          264    27  10.227273
0              автомобиль    161332-214622           543    57  10.497238
1              автомобиль    161332-214622           292    32  10.958904
                             98536-132131            256    29  11.328125
               образование   98536-132131            251    31  12.350598
               свадьба       20666-98536             149    20  13.422819
               автомобиль    132131-161332           332    46  13.855422

### Шаг 4. Общий вывод

Количество детей, цели кредита и общий доход, влияют на вероятность возврата кредита. 
Сводная таблица показывает что уровень просрочки меняется по группам от 4 до 14%. Что может быть основанием для построения скориинговой модели на основании этих параметров. 

Однако нужно учесть и тот факт что такие поля как пол, образование, тип занятости и семейное положение также имеют предсказательную силу. Их также нжуно использовать для построения модели одобрения.

---

<b>UPD: <font color='blue'>Хорошая доработка и отличный старт выполнения проектов. Впереди еще много интересных исследований. Успехов в дальнейшем прохождении курса!

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.